In [4]:
!pip install -q torch transformers datasets accelerate scikit-learn seqeval pandas matplotlib seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.8 MB/s eta 0:00:00


In [5]:
from google.colab import files
uploaded = files.upload()

Saving CLS_Dataset.csv to CLS_Dataset.csv


# BERT Classification Model

In [6]:
import os
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from sklearn.metrics import accuracy_score
import re

### Extracting data

In [7]:
filename = "CLS_Dataset.csv"
filepath = os.path.abspath(filename)
df = pd.read_csv(filepath)

df["text"] = df["text"].astype(str)

df = df[df["text"].notnull() & df["text"].str.strip() != ""]

df = df.drop_duplicates(subset=["text"])

def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9\s.,!?]', '', text)

df["text"] = df["text"].apply(clean_text)
df = df[df["text"].str.split().apply(len) >= 5]

dataset = Dataset.from_pandas(df)

dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 39722
})

### Creating Labels for Sentiments

In [8]:
labels_dict={0:"Negative",1:"Positive"}

## Class for BERT Classification

In [21]:
class BERTCLassification:

    def __init__(self, modelname, col_name, num_labels):
        self.modelname = modelname
        self.text = col_name
        self.num_labels = num_labels
        self.tokenizer = AutoTokenizer.from_pretrained(self.modelname)
        self.trainer = None

    def encoding_data(self, dataset, batched=True, batch_size=2000):
        def tokenize(batch):
            return self.tokenizer(batch[self.text],padding="max_length", truncation=True)
        return dataset.map(tokenize,batched=batched, batch_size=batch_size )

    def train(self, train, test, learning_rate, epochs):

        model = AutoModelForSequenceClassification.from_pretrained(self.modelname, num_labels=self.num_labels).to(device)

        training_args = TrainingArguments(
            output_dir='./results',
            eval_strategy="epoch",
            save_strategy="epoch",
            learning_rate= learning_rate,
            per_device_train_batch_size=64,
            per_device_eval_batch_size=64,
            num_train_epochs= epochs,
            weight_decay=0.01,
        )

        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = logits.argmax(axis=-1)
            return {"accuracy": accuracy_score(labels, predictions)}

        self.trainer = Trainer(
            model= model,
            args = training_args,
            train_dataset= train,
            eval_dataset= test,
            compute_metrics= compute_metrics
        )

        self.trainer.train()

    def evaluate(self):
        return self.trainer.evaluate()

    def save_model(self, path):
        self.trainer.save_model(path)
        self.tokenizer.save_pretrained(path)


SyntaxError: invalid syntax (<ipython-input-21-e4e6bee0d288>, line 24)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [22]:
bert_cls = BERTCLassification("distilbert-base-uncased","text",2)

tokenzied_data = bert_cls.encoding_data(dataset, batch_size=5000)

NameError: name 'bert_cls' is not defined

In [18]:
split = tokenzied_data.train_test_split(test_size=0.2,seed=42)
os.environ["WANDB_DISABLED"] = "true"

trained = bert_cls.train(split["train"],split['test'],2e-5,3)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.201044,0.923977
2,0.262100,0.189431,0.930522


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.201044,0.923977
2,0.262100,0.189431,0.930522
3,0.153200,0.202133,0.932914


In [20]:
result = bert_cls.evaluate()
result

{'eval_loss': 0.2021334171295166,
 'eval_accuracy': 0.9329137822529893,
 'eval_runtime': 131.8353,
 'eval_samples_per_second': 60.265,
 'eval_steps_per_second': 0.948,
 'epoch': 3.0}

In [ ]:
bert_cls.save_model("bert_cls_model")
!zip -r bert_cls_model.zip bert_cls_model
files.download("bert_cls_model.zip")